# Efficiency Simulation

In [1]:
from __future__ import annotations
from dataclasses import dataclass
from coordinates import Location, Checkpoint, device
import math as m
import copy
from collections.abc import Callable

from torch import FloatTensor, IntTensor, tensor
import torch
import torch.autograd.profiler as profiler
from tqdm.auto import tqdm

In [2]:
import pandas as pd
from coordinates import Checkpoint, Location, device
from torch import tensor

track_data = pd.read_csv("./sem_2023_us.csv")

track_data = track_data.rename(columns={
    "Metres above sea level": "Altitude"
})

track_data.head(10)

checkpoints: list[Checkpoint] = []
for i, row in track_data.iterrows():
    location = Location.construct(row["Latitude"], row["Longitude"], row["Altitude"])
    # print(f"Lat {row['Latitude']}, Long {row['Longitude']}, Alt {row['Altitude']}, x {location.x}, y {location.y}, z {location.z}")
    checkpoints.append(Checkpoint(location, location))

print(f"Found {len(checkpoints)} checkpoints")

Found 2696 checkpoints


In [3]:
from graph import Graph, Node, Transition 
def get_coefficient_of_drag(bearing: float) -> float:
    return 0.33

def get_projected_area(bearing: float) -> float:
    """Returns in mm^2"""
    return 943416

g = Graph.construct(
    checkpoints=checkpoints,
    n_points_per_checkpoint=1,
    max_velocity=42 * 1000 / 3600,  # Max velocity the car is allowed to go is 42 km/h
    velocity_step_size=7000 / 3600,
    wind_velocity=5000 / 3600,
    wind_bearing=200,
    mass=108000,
    coefficient_of_friction=0.03, # TODO: Figure this out
    get_coefficient_of_drag=get_coefficient_of_drag,
    get_projected_area=get_projected_area,
)

100%|██████████| 2695/2695 [01:18<00:00, 34.12it/s]


In [4]:
import random

def get_graph_distance(graph: Graph) -> float:
    '''Returns the length of a random path through the graph'''

    distance = 0
    cursor = graph.start
    while cursor is not None:
        if len(cursor.transitions) == 0:
            cursor = None
        else:
            idx = random.randint(0, len(cursor.transitions) - 1)
            distance += cursor.position.distance(cursor.transitions[idx].target.position)
            cursor = cursor.transitions[idx].target

    return distance

distance = get_graph_distance(g)
print(f"Graph Distance: {distance / 1000} km")


Graph Distance: 3.5690884038885002 km


In [5]:
def topological_order(graph: Graph) -> list[int]:
    '''Returns the ids in topo order'''
    # Topological sorting using DFS
    topological_order: list[int] = []
    marked_nodes: set[int] = set() # Used to track visited nodes

    def visit(node: Node):
        if node.id in marked_nodes:
            return

        for transition in node.transitions:
            visit(transition.target)

        marked_nodes.add(node.id)
        topological_order.insert(0, node.id)

    # Generate the topological order
    for node_id in graph.nodes.keys():
        if node_id not in marked_nodes:
            visit(graph.nodes[node_id])

    return topological_order

def cheapest_path(graph: Graph, work_weight: float, time_weight: float):
    # Topological sorting using DFS
    sorted_nodes = topological_order(graph)

    # Get all node ids
    node_ids = list(graph.nodes.keys())

    # Initialize cost and time dictionaries
    min_cost  = {node_id: float('inf') for node_id in node_ids}
    min_energy  = {node_id: float('inf') for node_id in node_ids}
    min_time = {node_id: float('inf') for node_id in node_ids}
    prev_node = {node_id: None for node_id in node_ids}

    min_cost[graph.start.id] = 0
    min_energy[graph.start.id] = 0
    min_time[graph.start.id] = 0

    # Iterate through nodes in topological order
    for node_id in sorted_nodes:
        current_node = graph.nodes[node_id]

        for transition in current_node.transitions:
            neighbor_id = transition.target.id
            new_time = min_time[node_id] + transition.time_required
            new_energy = min_energy[node_id] + transition.work_required

            # Calculate the new weighted cost
            weighted_cost = work_weight * new_energy + time_weight * new_time

            # Check if the new path offers a lower cost
            if weighted_cost < min_cost[neighbor_id]:
                min_cost[neighbor_id] = weighted_cost
                min_time[neighbor_id] = new_time
                min_energy[neighbor_id] = new_energy
                prev_node[neighbor_id] = node_id

    # Find the minimum cost path to the end node
    if min_cost[graph.end.id] == float('inf'):
        raise ValueError("No path found within reasonable time.")
    
    # Reconstruct the path
    path = []
    node_id = graph.end.id
    while node_id is not None:
        path.append(node_id)
        node_id = prev_node[node_id]

    path.reverse()  # Reverse to get the correct order from start to end
    return path, min_cost[graph.end.id], min_time[graph.end.id], min_energy[graph.end.id]

In [14]:
energy_weight = 1
time_weight = 100

node_ids, cost, time, energy = cheapest_path(g, energy_weight, time_weight)

distance = 0
last_node = node_ids[0]
for node in node_ids[1:]:
    a = g.get_node(last_node)
    b = g.get_node(node)

    if a is not None and b is not None:
        distance += a.position.distance(b.position)
    else:
        print("Skip")
    last_node = node

def kilo_watt_hour(joules: float):
    return joules / 3600000

effiency = (distance / 1000) / kilo_watt_hour(energy)

print(f"Energy {energy/1000} kJ | Energy {kilo_watt_hour(energy)} kWh | Time {time / 60} minutes | Cost {cost} | Efficiency {effiency}km/kWh | Distance: {distance / 1000} km")


Energy 707249.1954543946 kJ | Energy 196.45810984844292 kWh | Time 61.11986923836856 minutes | Cost 707285867.3759376 | Efficiency 0.018157576959688594km/kWh | Distance: 3.5672032489280583 km
